# Demand Forecasting with a Neural Network model

In this article we look into the algorithm of a vanilla Neural Network to forecast demand of 50 unique items that are sold in 10 stores by forecasting the sales. This Neural Network model is executed with Keras. We also look into feature engineering for time series analysis such as adding columns like day of week, month, is_month_end. These kinds of columns give us more information from a simple date column for us to understand and pick up demand patterns depending on the day it is. I have also added lag features to the target variable.

## Importing the data

In [1]:
#importing libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from time import time
from fastai.imports import *
from fastai.tabular.core import add_datepart
import preprocess

import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

This dataset is from the kaggle competion "Store Item Demand Forecasting Challenge", [here](https://www.kaggle.com/competitions/demand-forecasting-kernels-only)

Columns:
- date: date in the format Year-Month-day
- store: unique store ID
- item: unique item ID
- sales: number of units sold on that date

In [3]:
train_data= pd.read_csv('demand-forecasting-kernels-only/train.csv', parse_dates = ['date'])
train_data

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [4]:
test_data= pd.read_csv('demand-forecasting-kernels-only/test.csv', parse_dates = ['date'])
test_data

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1
...,...,...,...,...
44995,44995,2018-03-27,10,50
44996,44996,2018-03-28,10,50
44997,44997,2018-03-29,10,50
44998,44998,2018-03-30,10,50


In [5]:
#Concatinating the train and test data
data= pd.concat([train_data, test_data],sort= False)
data

,date,store,item,sales,id
0,2013-01-01,1,1,13.0,NaN
1,2013-01-02,1,1,11.0,NaN
2,2013-01-03,1,1,14.0,NaN
3,2013-01-04,1,1,13.0,NaN
4,2013-01-05,1,1,10.0,NaN
...,...,...,...,...,...
44995,2018-03-27,10,50,NaN,44995.0
44996,2018-03-28,10,50,NaN,44996.0
44997,2018-03-29,10,50,NaN,44997.0
44998,2018-03-30,10,50,NaN,44998.0


In [6]:
sub_file= pd.read_csv('demand-forecasting-kernels-only/sample_submission.csv')

## Looking into the data

The target variable is "sales"

In [7]:
data.shape

(958000, 5)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 958000 entries, 0 to 44999
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    958000 non-null  datetime64[ns]
 1   store   958000 non-null  int64         
 2   item    958000 non-null  int64         
 3   sales   913000 non-null  float64       
 4   id      45000 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 43.9 MB


In [9]:
data.isnull().sum()

date          0
store         0
item          0
sales     45000
id       913000
dtype: int64

In [10]:
data[data.duplicated()]

,date,store,item,sales,id


In [11]:
data['store'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

In [12]:
data['item'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
      dtype=int64)

In [13]:
#Finding the starting and ending date
print("Starting date: ", data.date.min())
print('Ending date: ', data.date.max())

Starting date:  2013-01-01 00:00:00
Ending date:  2018-03-31 00:00:00


There are 10 stores that sell 50 unique items. Lets look into each stores and items sale with some summary statistics.

In [14]:
data.groupby(["store"]).agg({"sales": ["count","sum", "mean", "median", "std", "min", "max"]})

sales                                                    
       count        sum       mean median        std  min    max
store                                                           
1      91300  4315603.0  47.268379   44.0  24.006252  1.0  155.0
2      91300  6120128.0  67.033165   62.0  33.595810  3.0  231.0
3      91300  5435144.0  59.530602   55.0  29.974102  3.0  196.0
4      91300  5012639.0  54.902946   51.0  27.733097  4.0  186.0
5      91300  3631016.0  39.770164   37.0  20.365757  2.0  130.0
6      91300  3627670.0  39.733516   37.0  20.310451  0.0  134.0
7      91300  3320009.0  36.363735   34.0  18.684825  1.0  122.0
8      91300  5856169.0  64.142048   60.0  32.231751  4.0  204.0
9      91300  5025976.0  55.049025   51.0  27.832186  4.0  195.0
10     91300  5360158.0  58.709288   54.0  29.554994  3.0  187.0

In [15]:
data.groupby(["item"]).agg({"sales": ["count","sum", "mean", "median", "std", "min", "max"]})

sales                                                     
      count        sum       mean median        std   min    max
item                                                            
1     18260   401384.0  21.981599   21.0   8.468922   1.0   59.0
2     18260  1069564.0  58.574151   56.0  20.093015   9.0  150.0
3     18260   669087.0  36.642223   35.0  13.179441   7.0  104.0
4     18260   401907.0  22.010241   21.0   8.403898   0.0   66.0
5     18260   335230.0  18.358708   18.0   7.265167   1.0   50.0
6     18260  1068281.0  58.503888   56.0  20.174898  11.0  148.0
7     18260  1068777.0  58.531051   56.0  20.146002  11.0  141.0
8     18260  1405108.0  76.950055   74.0  26.130697  15.0  181.0
9     18260   938379.0  51.389869   49.5  17.790158   6.0  134.0
10    18260  1337133.0  73.227437   70.0  24.823725  14.0  175.0
11    18260  1271925.0  69.656353   67.0  23.744732  11.0  170.0
12    18260  1271534.0  69.634940   67.0  23.738663  12.0  170.0
13    18260  1539621.0  84.316594   81.0  28.311031  20.0  210.0
14    18260  1071531.0  58.681873   56.0  20.079860  12.0  152.0
15    18260  1607442.0  88.030778   85.0  29.522852  17.0  231.0
16    18260   468480.0  25.656079   25.0   9.603270   2.0   70.0
17    18260   602486.0  32.994852   32.0  11.967610   4.0   83.0
18    18260  1538876.0  84.275794   81.0  28.430621  18.0  208.0
19    18260   736892.0  40.355531   39.0  14.332645   5.0   99.0
20    18260   867641.0  47.515936   46.0  16.490487   9.0  127.0
21    18260   736190.0  40.317087   39.0  14.338006   7.0  109.0
22    18260  1469971.0  80.502245   78.0  27.118163  14.0  214.0
23    18260   534979.0  29.297864   28.0  10.819549   3.0   81.0
24    18260  1205975.0  66.044633   64.0  22.531555  14.0  156.0
25    18260  1473334.0  80.686418   78.0  27.238817  18.0  193.0
26    18260   869981.0  47.644085   46.0  16.723912   8.0  119.0
27    18260   402628.0  22.049726   21.0   8.461641   1.0   59.0
28    18260  1604713.0  87.881325   85.0  29.501781  16.0  206.0
29    18260  1271240.0  69.618839   67.0  23.635631  15.0  173.0
30    18260   736554.0  40.337021   39.0  14.363331   5.0  115.0
31    18260  1070845.0  58.644304   57.0  20.104705  10.0  159.0
32    18260   803107.0  43.981763   42.0  15.574556   5.0  119.0
33    18260  1270183.0  69.560953   67.0  23.718343  15.0  169.0
34    18260   469935.0  25.735761   25.0   9.617910   2.0   79.0
35    18260  1201541.0  65.801807   63.0  22.461990  12.0  168.0
36    18260  1406548.0  77.028916   74.0  26.067440  16.0  188.0
37    18260   534258.0  29.258379   28.0  10.771547   3.0   74.0
38    18260  1470330.0  80.521906   77.0  27.141799  15.0  188.0
39    18260   801311.0  43.883406   42.0  15.511550   7.0  112.0
40    18260   534094.0  29.249398   28.0  10.822959   3.0   74.0
41    18260   401759.0  22.002136   21.0   8.402470   2.0   60.0
42    18260   669925.0  36.688116   35.0  13.215112   5.0   96.0
43    18260   936635.0  51.294359   49.0  17.801008   9.0  126.0
44    18260   536811.0  29.398193   28.0  10.797738   3.0   78.0
45    18260  1471467.0  80.584173   78.0  27.318402  18.0  205.0
46    18260  1070764.0  58.639869   56.0  20.220879  11.0  150.0
47    18260   401781.0  22.003341   21.0   8.420102   2.0   61.0
48    18260   937703.0  51.352848   49.0  17.881917   8.0  130.0
49    18260   535663.0  29.335323   28.0  10.874788   3.0   77.0
50    18260  1203009.0  65.882202   63.0  22.416031  12.0  164.0

In [16]:
#Making sure that each store sells each item
data.groupby(['store'])['item'].nunique()

store
1     50
2     50
3     50
4     50
5     50
6     50
7     50
8     50
9     50
10    50
Name: item, dtype: int64

In [17]:
train_data.store.nunique()

10

In [18]:
train_data.date.nunique()

1826

Looking at the data we can see that there is one row for the number of units sold for each day at each store for each item resulting in 1826*10*50 = 913000. There are 500 different time series data in the train data set.

## Feature Engineering

In [19]:
train_date = train_data['date']
test_date= test_data['date']

We use the add_datepart() function from the fastai library to add columns:
- Year
- Month
- Week
- Day
- Dayofweek
- Dayofyear
- Is_month_end
- Is_month_start
- Is_quarter_end
- Is_quarter_start
- Is_year_end
- Is_year_start

In [20]:
add_datepart(train_data, 'date')
add_datepart(test_data, 'date')

,id,store,item,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,1,1,2018,1,1,1,0,1,False,True,False,True,False,True,1.514765e+09
1,1,1,1,2018,1,1,2,1,2,False,False,False,False,False,False,1.514851e+09
2,2,1,1,2018,1,1,3,2,3,False,False,False,False,False,False,1.514938e+09
3,3,1,1,2018,1,1,4,3,4,False,False,False,False,False,False,1.515024e+09
4,4,1,1,2018,1,1,5,4,5,False,False,False,False,False,False,1.515110e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,44995,10,50,2018,3,13,27,1,86,False,False,False,False,False,False,1.522109e+09
44996,44996,10,50,2018,3,13,28,2,87,False,False,False,False,False,False,1.522195e+09
44997,44997,10,50,2018,3,13,29,3,88,False,False,False,False,False,False,1.522282e+09
44998,44998,10,50,2018,3,13,30,4,89,False,False,False,False,False,False,1.522368e+09


In [21]:
train_data

,store,item,sales,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,1,1,13,2013,1,1,1,1,1,False,True,False,True,False,True,1.356998e+09
1,1,1,11,2013,1,1,2,2,2,False,False,False,False,False,False,1.357085e+09
2,1,1,14,2013,1,1,3,3,3,False,False,False,False,False,False,1.357171e+09
3,1,1,13,2013,1,1,4,4,4,False,False,False,False,False,False,1.357258e+09
4,1,1,10,2013,1,1,5,5,5,False,False,False,False,False,False,1.357344e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912995,10,50,63,2017,12,52,27,2,361,False,False,False,False,False,False,1.514333e+09
912996,10,50,59,2017,12,52,28,3,362,False,False,False,False,False,False,1.514419e+09
912997,10,50,74,2017,12,52,29,4,363,False,False,False,False,False,False,1.514506e+09
912998,10,50,62,2017,12,52,30,5,364,False,False,False,False,False,False,1.514592e+09


We add the date column back because the add_datepart deletes it

In [22]:
train_data['date']= train_date
test_data['date']= test_date

In [23]:
data= pd.concat([train_data, test_data], sort= False)
data

,store,item,sales,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,date,id
0,1,1,13.0,2013,1,1,1,1,1,False,True,False,True,False,True,1.356998e+09,2013-01-01,NaN
1,1,1,11.0,2013,1,1,2,2,2,False,False,False,False,False,False,1.357085e+09,2013-01-02,NaN
2,1,1,14.0,2013,1,1,3,3,3,False,False,False,False,False,False,1.357171e+09,2013-01-03,NaN
3,1,1,13.0,2013,1,1,4,4,4,False,False,False,False,False,False,1.357258e+09,2013-01-04,NaN
4,1,1,10.0,2013,1,1,5,5,5,False,False,False,False,False,False,1.357344e+09,2013-01-05,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,10,50,NaN,2018,3,13,27,1,86,False,False,False,False,False,False,1.522109e+09,2018-03-27,44995.0
44996,10,50,NaN,2018,3,13,28,2,87,False,False,False,False,False,False,1.522195e+09,2018-03-28,44996.0
44997,10,50,NaN,2018,3,13,29,3,88,False,False,False,False,False,False,1.522282e+09,2018-03-29,44997.0
44998,10,50,NaN,2018,3,13,30,4,89,False,False,False,False,False,False,1.522368e+09,2018-03-30,44998.0


Lag features are the values of vales with lags in them. For example a lag feature with a lag of 7 would be a column of what the sales were 7 days ago. Time series data has seasonality and trends and this is a way to factor in the lags for different periods such as 91 (three months), 98 (three months and a week ago) and so on

In [24]:
def lag_features(df, lags):
    for lag in lags:
        df['sales_lag_' + str(lag)] = df.groupby(["store", "item"])['sales'].transform(lambda x: x.shift(lag))
        df.fillna(0, inplace=True)
    return df

In [25]:
lag_features(data, [91, 98, 105, 112, 119, 126, 182, 364, 546, 728])
data

,store,item,sales,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,...,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728
0,1,1,13.0,2013,1,1,1,1,1,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,11.0,2013,1,1,2,2,2,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,14.0,2013,1,1,3,3,3,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,13.0,2013,1,1,4,4,4,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,10.0,2013,1,1,5,5,5,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,10,50,0.0,2018,3,13,27,1,86,False,...,41.0,54.0,67.0,68.0,68.0,81.0,79.0,60.0,97.0,73.0
44996,10,50,0.0,2018,3,13,28,2,87,False,...,63.0,51.0,67.0,60.0,76.0,81.0,80.0,73.0,80.0,68.0
44997,10,50,0.0,2018,3,13,29,3,88,False,...,59.0,63.0,72.0,66.0,73.0,87.0,82.0,68.0,99.0,75.0
44998,10,50,0.0,2018,3,13,30,4,89,False,...,74.0,75.0,72.0,67.0,66.0,81.0,90.0,69.0,79.0,83.0


In [26]:
data.columns

Index(['store', 'item', 'sales', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed', 'date',
       'id', 'sales_lag_91', 'sales_lag_98', 'sales_lag_105', 'sales_lag_112',
       'sales_lag_119', 'sales_lag_126', 'sales_lag_182', 'sales_lag_364',
       'sales_lag_546', 'sales_lag_728'],
      dtype='object')

In [27]:
#Converting the boolean columns to integer columns
bol_col = ['Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'Is_year_end', 'Is_year_start']

for col in bol_col:
    data[col] = data[col]*1

In [28]:
del data['Dayofyear']
del data['Elapsed']
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 958000 entries, 0 to 44999
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   store             958000 non-null  int64         
 1   item              958000 non-null  int64         
 2   sales             958000 non-null  float64       
 3   Year              958000 non-null  int64         
 4   Month             958000 non-null  int64         
 5   Week              958000 non-null  int64         
 6   Day               958000 non-null  int64         
 7   Dayofweek         958000 non-null  int64         
 8   Is_month_end      958000 non-null  int32         
 9   Is_month_start    958000 non-null  int32         
 10  Is_quarter_end    958000 non-null  int32         
 11  Is_quarter_start  958000 non-null  int32         
 12  Is_year_end       958000 non-null  int32         
 13  Is_year_start     958000 non-null  int32         
 14  date 

In [29]:
data.shape

(958000, 26)

We have to one hot encode the categorical variables with the get_dummies() pandas function

In [30]:
data = pd.get_dummies(data, columns=['store', 'item', "Week", 'Month', 'Day', "Year"])
data.shape

(958000, 182)

We scale the sales by taking the log value of it.

In [31]:
data['sales']= np.log(data['sales'])

In [32]:
df_model = data.sort_values("date").reset_index(drop = True)

In [34]:
#Splitting the train dataset into train and validation
train= df_model.loc[(df_model["date"] <  "2017-01-01"), :]


val=   df_model.loc[(df_model["date"] >= "2017-01-01") & (df_model["date"] < "2017-04-01"), :]


In [35]:
train

,sales,Dayofweek,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,date,id,...,Day_28,Day_29,Day_30,Day_31,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017,Year_2018
0,2.564949,1,0,1,0,1,0,1,2013-01-01,0.0,...,0,0,0,0,1,0,0,0,0,0
1,2.772589,1,0,1,0,1,0,1,2013-01-01,0.0,...,0,0,0,0,1,0,0,0,0,0
2,2.995732,1,0,1,0,1,0,1,2013-01-01,0.0,...,0,0,0,0,1,0,0,0,0,0
3,3.526361,1,0,1,0,1,0,1,2013-01-01,0.0,...,0,0,0,0,1,0,0,0,0,0
4,2.944439,1,0,1,0,1,0,1,2013-01-01,0.0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730495,4.043051,5,1,0,1,0,1,0,2016-12-31,0.0,...,0,0,0,1,0,0,0,1,0,0
730496,3.583519,5,1,0,1,0,1,0,2016-12-31,0.0,...,0,0,0,1,0,0,0,1,0,0
730497,3.332205,5,1,0,1,0,1,0,2016-12-31,0.0,...,0,0,0,1,0,0,0,1,0,0
730498,4.007333,5,1,0,1,0,1,0,2016-12-31,0.0,...,0,0,0,1,0,0,0,1,0,0


In [36]:
val

,sales,Dayofweek,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,date,id,...,Day_28,Day_29,Day_30,Day_31,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017,Year_2018
730500,3.637586,6,0,1,0,1,0,1,2017-01-01,0.0,...,0,0,0,0,0,0,0,0,1,0
730501,4.672829,6,0,1,0,1,0,1,2017-01-01,0.0,...,0,0,0,0,0,0,0,0,1,0
730502,4.060443,6,0,1,0,1,0,1,2017-01-01,0.0,...,0,0,0,0,0,0,0,0,1,0
730503,3.258097,6,0,1,0,1,0,1,2017-01-01,0.0,...,0,0,0,0,0,0,0,0,1,0
730504,3.332205,6,0,1,0,1,0,1,2017-01-01,0.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775495,4.394449,4,1,0,1,0,0,0,2017-03-31,0.0,...,0,0,0,1,0,0,0,0,1,0
775496,3.526361,4,1,0,1,0,0,0,2017-03-31,0.0,...,0,0,0,1,0,0,0,0,1,0
775497,3.761200,4,1,0,1,0,0,0,2017-03-31,0.0,...,0,0,0,1,0,0,0,0,1,0
775498,3.970292,4,1,0,1,0,0,0,2017-03-31,0.0,...,0,0,0,1,0,0,0,0,1,0


In [37]:
cols = [col for col in train.columns if col not in ['date', 'id', "sales", "Year"]]

In [38]:
X_train = train[cols]
Y_train = train['sales']

X_val = val[cols]
Y_val = val['sales']

X_train.shape, Y_train.shape, X_val.shape, Y_val.shape

((730500, 179), (730500,), (45000, 179), (45000,))

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from keras import metrics
import tensorflow as tf

Neural Network architecture:
- relu is used as the activation function
- Three Hidden layers
- One dropout layer
- Adam optimizer

In [40]:
NN_model = Sequential()

NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))#The Input Layer

NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))#The Hidden Layers
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))#The Hidden Layers
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))#The Hidden Layers

NN_model.add(Dropout(0.2))#Dropout layer

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))#The Output Layer

#Compile the network
NN_model.compile(loss=tf.keras.losses.mae, optimizer="adam", metrics=['mae'])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               23040     
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 187,905
Trainable params: 187,905
Non-trai

In [41]:
#Define a checkpoint callback :
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [42]:
#Train the model
NN_model.fit(X_train.values, Y_train.values, epochs=2, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/2
18241/18263 [============================>.] - ETA: 0s - loss: inf - mae: inf
Epoch 1: val_loss improved from inf to 0.13054, saving model to Weights-001--0.13054.hdf5
18263/18263 [==============================] - 46s 2ms/step - loss: inf - mae: inf - val_loss: 0.1305 - val_mae: 0.1305
Epoch 2/2
18249/18263 [============================>.] - ETA: 0s - loss: inf - mae: inf
Epoch 2: val_loss improved from 0.13054 to 0.12408, saving model to Weights-002--0.12408.hdf5
18263/18263 [==============================] - 47s 3ms/step - loss: inf - mae: inf - val_loss: 0.1241 - val_mae: 0.1241


In [43]:
# SMAPE: Symmetric mean absolute percentage error (adjusted MAPE)
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds-target)
    denom = np.abs(preds)+np.abs(target)
    smape_val = (200*np.sum(num/denom))/n
    return smape_val

In [44]:
print("VALID SMAPE:", smape(np.expm1(NN_model.predict(X_val).flatten()), np.expm1(Y_val)))

VALID SMAPE: 14.376395958509391


In [45]:
train_predictions = NN_model.predict(X_train).flatten()
train_results = pd.DataFrame(data={'Train Predictions':np.expm1(train_predictions), 'Actuals':np.expm1(Y_train)})
train_results

,Train Predictions,Actuals
0,9.130653,12.0
1,18.645594,15.0
2,22.092216,19.0
3,36.266087,33.0
4,14.724173,18.0
...,...,...
730495,48.770115,56.0
730496,35.707886,35.0
730497,26.677916,27.0
730498,48.472633,54.0


In [46]:
df_final_model= data.copy()
train = df_final_model.loc[(df_final_model["date"] <  "2018-01-01"), :]
Y_train = train['sales']
X_train = train[cols]

test = df_final_model.loc[(df_final_model["date"] >=  "2018-01-01"), :]
X_test = test[cols]

In [47]:
df_final_model

,sales,Dayofweek,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,date,id,...,Day_28,Day_29,Day_30,Day_31,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017,Year_2018
0,2.564949,1,0,1,0,1,0,1,2013-01-01,0.0,...,0,0,0,0,1,0,0,0,0,0
1,2.397895,2,0,0,0,0,0,0,2013-01-02,0.0,...,0,0,0,0,1,0,0,0,0,0
2,2.639057,3,0,0,0,0,0,0,2013-01-03,0.0,...,0,0,0,0,1,0,0,0,0,0
3,2.564949,4,0,0,0,0,0,0,2013-01-04,0.0,...,0,0,0,0,1,0,0,0,0,0
4,2.302585,5,0,0,0,0,0,0,2013-01-05,0.0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,-inf,1,0,0,0,0,0,0,2018-03-27,44995.0,...,0,0,0,0,0,0,0,0,0,1
44996,-inf,2,0,0,0,0,0,0,2018-03-28,44996.0,...,1,0,0,0,0,0,0,0,0,1
44997,-inf,3,0,0,0,0,0,0,2018-03-29,44997.0,...,0,1,0,0,0,0,0,0,0,1
44998,-inf,4,0,0,0,0,0,0,2018-03-30,44998.0,...,0,0,1,0,0,0,0,0,0,1


In [48]:
X_train

,Dayofweek,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,sales_lag_91,sales_lag_98,sales_lag_105,...,Day_28,Day_29,Day_30,Day_31,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017,Year_2018
0,1,0,1,0,1,0,1,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,2,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,3,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
3,4,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,5,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912995,2,0,0,0,0,0,0,80.0,72.0,84.0,...,0,0,0,0,0,0,0,0,1,0
912996,3,0,0,0,0,0,0,82.0,89.0,90.0,...,1,0,0,0,0,0,0,0,1,0
912997,4,0,0,0,0,0,0,90.0,97.0,94.0,...,0,1,0,0,0,0,0,0,1,0
912998,5,0,0,0,0,0,0,103.0,97.0,97.0,...,0,0,1,0,0,0,0,0,1,0


In [49]:
test_predictions = NN_model.predict(X_test).flatten()

In [50]:
test_results = pd.DataFrame(data={'Test Predictions':np.expm1(test_predictions)})
test_results

,Test Predictions
0,11.439732
1,11.893713
2,12.358550
3,15.164682
4,16.545284
...,...
44995,67.531662
44996,70.649574
44997,73.486961
44998,78.060699
